# Database Writes

Apache Spark&trade; and Azure Databricks&reg; allow you to write to a number of target databases in parallel, storing the transformed data from from your ETL job.

-sandbox
### Database Writes in Spark

Writing to a database in Spark differs from other tools largely due to its distributed nature. There are a number of variables that can be tweaked to optimize performance, largely relating to how data is organized on the cluster. Partitions are the first step in understanding performant database connections.

**A partition is a portion of your total data set,** which is divided into many of these portions so Spark can distribute your work across a cluster. 

The other concept needed to understand Spark's computation is a slot (also known as a core). **A slot/core is a resource available for the execution of computation in parallel.** In brief, a partition refers to the distribution of data while a slot refers to the distribution of computation.

<div><img src="https://files.training.databricks.com/images/eLearning/ETL-Part-2/partitions-and-cores.png" style="height: 400px; margin: 20px"/></div>

As a general rule of thumb, the number of partitions should be a multiple of the number of cores. For instance, with 5 partitions and 8 slots, 3 of the slots will be underutilized. With 9 partitions and 8 slots, a job will take twice as long as it waits for the extra partition to finish.

### Managing Partitions

In the context of JDBC database writes, **the number of partitions determine the number of connections used to push data through the JDBC API.** There are two ways to controll this parallelism:  

| Function | Transformation Type | Use | Evenly distributes data across partitions? |
| :----------------|:----------------|:----------------|:----------------| 
| `.coalesce(n)`   | narrow (does not shuffle data) | reduce the number of partitions | no |
| `.repartition(n)`| wide (includes a shuffle operation) | increase the number of partitions | yes |

Run the cell below to mount the data.

In [5]:
%run "./Includes/Classroom-Setup"

Start by importing a DataFrame of Wikipedia pageviews.

In [7]:
wikiDF = (spark.read
  .parquet("/mnt/training/wikipedia/pageviews/pageviews_by_second.parquet")
)
display(wikiDF)

View the number of partitions by changing the DataFrame into an RDD and using the `.getNumPartitions()` method.  
Since the Parquet file was saved with 5 partitions, those paritions are retained when you import the data.

In [9]:
partitions = wikiDF.rdd.getNumPartitions()
print("Partitions: {0:,}".format( partitions ))

To increase the number of partitions to 16, use `.repartition()`.

In [11]:
repartitionedWikiDF = wikiDF.repartition(16)
print("Partitions: {0:,}".format( repartitionedWikiDF.rdd.getNumPartitions() ))

To reduce the number of partitions, use `.coalesce()`.

In [13]:
coalescedWikiDF = repartitionedWikiDF.coalesce(2)
print("Partitions: {0:,}".format( coalescedWikiDF.rdd.getNumPartitions() ))

-sandbox
### Configure Default Partitions

Spark uses a default value of 200 partitions, which comes from real-world experience by Databricks engineers. This is an adjustable configuration setting. Run the following cell to see this value.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Get and set any number of different configuration settings in this manner. <a href="https://spark.apache.org/docs/latest/configuration.html" target="_blank">See the Spark documents</a> for details.

In [15]:
spark.conf.get("spark.sql.shuffle.partitions")

Adjust the number of partitions with the following cell.  **This changes the number of partitions after a shuffle operation.**

In [17]:
spark.conf.set("spark.sql.shuffle.partitions", "8")

Now check to see how this changes an operation involving a data shuffle, such as an `.orderBy()`.  Recall that coalesced `coalescedWikiDF` to 2 partitions.

In [19]:
orderByPartitions = coalescedWikiDF.orderBy("requests").rdd.getNumPartitions()
print("Partitions: {0:,}".format( orderByPartitions ))

The `.orderBy()` triggered the repartition of the DataFrame into 8 partitions.  Now reset the default value.

In [21]:
spark.conf.set("spark.sql.shuffle.partitions", "200")

### Parallel Database Writes

Database writes are the inverse of what was covered in Lesson 4 of ETL Part 1.  In that lesson you defined the number of partitions in the call to the database.  

**By contrast and when writing to a database, the number of active connections to the database is determined by the number of partitions of the DataFrame.**

Examine this by writing `wikiDF` to the `/tmp` directory.  Recall that `wikiDF` has 5 partitions.

In [24]:
wikiDF.write.mode("OVERWRITE").parquet("/tmp/wiki.parquet")

Examine the number of paritions in `/tmp/wiki.parquet`.

In [26]:
%fs ls /tmp/wiki.parquet

This file has 5 parts, meaning Spark wrote the data through 5 different connections to this directory in the file system.

### Examining in the Spark UI

Click the arrow next to `Spark Jobs` under the following code cell in order to see a breakdown of the job you triggered. Click the next arrow to see a breakdown of the stages.

In [29]:
wikiDF.repartition(12).write.mode("OVERWRITE").parquet("/tmp/wiki.parquet")

5 stages were initially triggered, one for each partition of our data.  When you repartitioned the DataFrame to 12 paritions, 12 stages were needed, one to write each partition of the data.  Run the following and observe how the repartitioning changes the number of stages.

In [31]:
wikiDF.repartition(10).write.mode("OVERWRITE").parquet("/tmp/wiki.parquet")

For more details, click `View` to examine the more details about the operation in the Spark UI.

### A Note on Upserts

Upserts insert a record into a database if it doesn't already exist, and updates the existing record if it does.  **Upserts are not supported in core Spark.** You can only append or overwrite.  Databricks offers a data management system called Databricks Delta that does allow for upserts and other transactional functionality. [See the Databricks Delta docs for more information.](https://docs.azuredatabricks.net/delta/index.html#delta-guide)

## Exercise 1: Changing Paritions

Change the number of partitions to prepare the optimal database write.

### Step 1: Import Helper Functions and Data

A function is defined for you to print out the number of records in each DataFrame.  Run the following cell to define that function.

In [36]:
def printRecordsPerPartition(df):
  '''
  Utility method to count & print the number of records in each partition
  '''
  print("Per-Partition Counts:")
  
  def countInPartition(iterator): 
    yield __builtin__.sum(1 for _ in iterator)
    
  results = (df.rdd                   # Convert to an RDD
    .mapPartitions(countInPartition)  # For each partition, count
    .collect()                        # Return the counts to the driver
  )

  for result in results: 
    print("* " + str(result))

Import the data to sitting in `/mnt/training/wikipedia/pageviews/pageviews_by_second.parquet` to `wikiDF`.

In [38]:
# TODO
wikiDF = # FILL_IN

In [39]:
# TEST - Run this cell to test your solution
dbTest("ET2-P-06-01-01", 7200000, wikiDF.count())
dbTest("ET2-P-06-01-02", ['timestamp', 'site', 'requests'], wikiDF.columns)

print("Tests passed!")

Print the count of records by partition using `printRecordsPerPartition()`.

In [41]:
printRecordsPerPartition(wikiDF)

### Step 2: Repartition the Data

Define three new DataFrames:

* `wikiDF1Partition`: `wikiDF` with 1 partition
* `wikiDF16Partition`: `wikiDF` with 16 partitions
* `wikiDF128Partition`: `wikiDF` with 128 partitions

In [43]:
# TODO
wikiDF1Partition = # FILL_IN
wikiDF16Partition = # FILL_IN
wikiDF128Partition = # FILL_IN

In [44]:
# TEST - Run this cell to test your solution

dbTest("ET2-P-06-02-01", 1, wikiDF1Partition.rdd.getNumPartitions())
dbTest("ET2-P-06-02-02", 16, wikiDF16Partition.rdd.getNumPartitions())
dbTest("ET2-P-06-02-03", 128, wikiDF128Partition.rdd.getNumPartitions())

print("Tests passed!")

### Step 3: Examine the Distribution of Records

Use `printRecordsPerPartition()` to examine the distribution of records across the partitions.

In [46]:
# TODO
printRecordsPerPartition( <FILL_IN> )

### Step 4: Coalesce `wikiDF16Partition` and Examine the Results

Coalesce `wikiDF16Partition` to `10` partitions, saving the result to `wikiDF16PartitionCoalesced`.

In [48]:
# TODO
wikiDF16PartitionCoalesced = # FILL_IN

In [49]:
# TEST - Run this cell to test your solution

dbTest("ET2-P-06-03-01", 10, wikiDF16PartitionCoalesced.rdd.getNumPartitions())

print("Tests passed!")

Examine the new distribution of data using `printRecordsPerPartition`.  Is the distribution uniform?

In [51]:
# TODO

## Review
**Question:** How do you determine the number of connections to the database you write to?  
**Answer:** Spark makes one connection for each partition in your data. Increasing the number of partitions increases the database connections.

**Question:** How do you increase and decrease the number of partitions in your data?  
**Answer:** `.repartitions(n)` increases the number of partitions in your data. It can also decrease the number of partitions, but since this is a wide operation it should be used sparingly.  `.coalesce(n)` decreases the number of partitions in your data. If you use `.coalesce(n)` with a number greater than the current paritions, this DataFrame method will have no effect.

**Question:** How can you change the default number of partitions?  
**Answer:** Changing the configuration parameter `spark.sql.shuffle.partitions` will alter the default number of partitions.

## Next Steps

Start the next lesson, [Table Management]($./07-Table-Management ).

## Additional Topics & Resources

**Q:** Where can I find more information on reading the Spark UI?  
**A:** Check out the Databricks blog on <a href="https://databricks.com/blog/2015/06/22/understanding-your-spark-application-through-visualization.html" target="_blank">Understanding your Spark Application through Visualization</a>